# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
#Your code here
import requests

In [2]:
api_key = 'Au0C3R2kALu3iJUl2OdlrtSzzER6VyXYn-D6jzU9x6vf43DghQvTKRVjJ6b7fDc04d-QYrLNCUCRJBwllnP8yvC0L4jxCugrWR-9o481ZMKXtpgTFokMv-zxfT2UXHYx'

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'location': 'NYC',
                'term' : 'pizza',
                'limit' : 50,
                'price' : "1,2,3,4",
                'open_now' : True
            }
response = requests.get(url, headers=headers, params=url_params)

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [3]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [4]:
# Your code here; use a function or loop to retrieve all the results from your original request
for key in response.json().keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    print('\n\n') #Seperate out data

businesses
<class 'list'>



total
<class 'int'>



region
<class 'dict'>





In [5]:
response.json()['businesses'][:2]

[{'id': 'WIhm0W9197f_rRtDziq5qQ',
  'alias': 'lombardis-pizza-new-york-4',
  'name': "Lombardi's Pizza",
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/UZ6V_hobp1KpYDPOHNoCKw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/lombardis-pizza-new-york-4?adjust_creative=IsyCUylDgXB-U-lS838dTQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=IsyCUylDgXB-U-lS838dTQ',
  'review_count': 5800,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
  'rating': 4.0,
  'coordinates': {'latitude': 40.72164, 'longitude': -73.9957},
  'transactions': ['delivery', 'pickup'],
  'price': '$$',
  'location': {'address1': '32 Spring St',
   'address2': '',
   'address3': '',
   'city': 'New York',
   'zip_code': '10012',
   'country': 'US',
   'state': 'NY',
   'display_address': ['32 Spring St', 'New York, NY 10012']},
  'phone': '+12129417994',
  'display_phone': '(212) 941-7994',
  'distance': 1804.4733931798112},
 {'id': 'zj8Lq1T8KIC5zwFief15jg',
  'alias

In [6]:
response.json()['total']

4200

In [7]:
response.json()['region']

{'center': {'longitude': -73.99429321289062, 'latitude': 40.70544486444615}}

## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [8]:
#Your code here
import pandas as pd

df = pd.DataFrame.from_dict(response.json()['businesses'])
print(len(df)) #Print how many rows
print(df.columns) #Print column names
df.head() #Previews the first five rows.

50
Index(['alias', 'categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')


,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,lombardis-pizza-new-york-4,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.72164, 'longitude': -73.9957}",(212) 941-7994,1804.473393,WIhm0W9197f_rRtDziq5qQ,https://s3-media1.fl.yelpcdn.com/bphoto/UZ6V_h...,False,"{'address1': '32 Spring St', 'address2': '', '...",Lombardi's Pizza,+12129417994,$$,4.0,5800,"[delivery, pickup]",https://www.yelp.com/biz/lombardis-pizza-new-y...
1,prince-street-pizza-new-york-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.7229087705545, 'longitude': -7...",(212) 966-4100,1941.963817,zj8Lq1T8KIC5zwFief15jg,https://s3-media3.fl.yelpcdn.com/bphoto/ZAukOy...,False,"{'address1': '27 Prince St', 'address2': None,...",Prince Street Pizza,+12129664100,$,4.5,2599,"[delivery, pickup]",https://www.yelp.com/biz/prince-street-pizza-n...
2,joes-pizza-new-york-4,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7305599, 'longitude': -74.0021...",(212) 366-1182,2873.553223,uc5qQMzs96rzjK27epDCug,https://s3-media2.fl.yelpcdn.com/bphoto/FhbFkr...,False,"{'address1': '7 Carmine St', 'address2': '', '...",Joe's Pizza,+12123661182,$,4.0,2206,[],https://www.yelp.com/biz/joes-pizza-new-york-4...
3,bleecker-street-pizza-new-york,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.7320627016512, 'longitude': -7...",(212) 924-4466,3072.954045,oJ_XSPZFpvIt6rvheSQl7A,https://s3-media4.fl.yelpcdn.com/bphoto/Kyi2tP...,False,"{'address1': '69 7th Ave S', 'address2': None,...",Bleecker Street Pizza,+12129244466,$,4.0,2007,"[delivery, pickup]",https://www.yelp.com/biz/bleecker-street-pizza...
4,l-industrie-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.71162, 'longitude': -73.95783}",(718) 599-0002,3145.016041,v1DHGRNCH9247WLYoaoA9A,https://s3-media2.fl.yelpcdn.com/bphoto/SRL_Rz...,False,"{'address1': '254 S 2nd St', 'address2': '', '...",L'industrie Pizzeria,+17185990002,$,5.0,402,"[delivery, pickup]",https://www.yelp.com/biz/l-industrie-pizzeria-...


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [9]:
#Your code here
! pip install folium
import folium


lat_long = df['coordinates'].iloc[0]
lat = lat_long['latitude']
long = lat_long['longitude']
yelp_map = folium.Map([lat, long])

for row in df.index:
    try:
        lat_long = df['coordinates'][row]
        lat = lat_long['latitude']
        long = lat_long['longitude']
        name = df['name'][row]
        rating = df['rating'][row]
        price = df['price'][row]
        details = "{}\nPrice: {} Rating:{}".format(name,str(price),str(rating))
        popup = folium.Popup(details, parse_html=True)
        marker = folium.Marker([lat, long], popup=popup)
        marker.add_to(yelp_map)
    except:
        print('Hit error on row: {}'.format(row))
yelp_map

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!